In [1]:
# import os

# # 设置环境变量
# os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

# # 打印环境变量以确认设置成功
# print(os.environ.get('HF_ENDPOINT'))

import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [2]:
import os
import subprocess
import json
import random
import re
import torch  # 新增
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig # 新增
from huggingface_hub import login # 新增
from sklearn.metrics import accuracy_score, classification_report

In [3]:
# ==========================================
# 3. 加载本地 Llama-3.1-8B-Instruct 模型
# ==========================================
MODEL_ID = "meta-llama/Llama-3.1-8B-Instruct"
HF_TOKEN = "" # 使用你之前提供的 Token

print(f"Loading model: {MODEL_ID}...")

# 1. 登录 Hugging Face (解决 403 权限问题)
login(token=HF_TOKEN)

# 2. 加载 Tokenizer 和 Model
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
    
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        torch_dtype=torch.float16, # 使用半精度节省显存
        device_map="auto",         # 自动分配到 GPU
    )
    
    # 设定 Llama 3 的终止符
    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]
    
    print("Model loaded successfully.")
    
except Exception as e:
    print(f"Failed to load model. Error: {e}")
    # 如果显存不够，可以尝试把 torch_dtype 改为 torch.bfloat16 (如果是30/40系显卡)
    # 或者尝试 4bit 量化 (需要 pip install bitsandbytes)

Loading model: meta-llama/Llama-3.1-8B-Instruct...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Model loaded successfully.


In [9]:
# ==========================================
# 2. 数据准备 (加载、采样、打乱)
# ==========================================
print("Loading dataset...")
try:
    # 尝试加载数据集
    protein_data = load_dataset('dnagpt/biopaws', 'protein_pair_short')
    #local_dataset_path = "./gene_lan_transfer" 
    # 修改这里：第一个参数改为本地路径
    # protein_data = load_dataset(
    #     local_dataset_path,          # 👈 这里改成你的本地文件夹路径
    #     'protein_sim_pair_450bp',    # 配置名保持不变
    #     trust_remote_code=True       # 👈 加上这个，允许执行本地文件夹里的加载脚本
    # )
    
    ds = protein_data['train']
    
    # 分离数据
    data_label_0 = [item for item in ds if item['label'] == 0]
    data_label_1 = [item for item in ds if item['label'] == 1]
    
    # 随机采样 (各50个)
    random.seed(42)
    sample_num = 50  # 注意：总数是100，所以各取50
    sampled_0 = random.sample(data_label_0, sample_num)
    sampled_1 = random.sample(data_label_1, sample_num)
    
    # 合并并打乱
    combined_data = sampled_0 + sampled_1
    random.shuffle(combined_data)
    
    print(f"Data prepared: {len(combined_data)} pairs.")

except Exception as e:
    print(f"Error loading dataset: {e}")
    
# 构建用于 Prompt 的 JSON List，并建立 ID -> Label 的映射用于后续验证
prompt_data_list = []
id_to_ground_truth = {}

for idx, item in enumerate(combined_data, 1):
    prompt_data_list.append({
        "id": idx,
        "seq_a": item['sentence1'],
        "seq_b": item['sentence2']
    })
    id_to_ground_truth[idx] = item['label']

Loading dataset...
Data prepared: 100 pairs.


In [10]:
# # ==========================================
# # 3. 存储数据到文件 (包含 Label)
# # ==========================================
# import json

# # 定义文件名
# output_filename = "prompt_data_list.json"

# print(f"Saving {len(prompt_data_list)} items to {output_filename}...")

# # --- 关键修改：在保存前，把 label 从 id_to_ground_truth 中合并进去 ---
# # 这一步不需要动前面的代码，直接利用已有的 id_to_ground_truth 字典
# for item in prompt_data_list:
#     item_id = item['id']
#     if item_id in id_to_ground_truth:
#         item['label'] = id_to_ground_truth[item_id]

# # 写入文件
# with open(output_filename, "w", encoding="utf-8") as f:
#     # indent=2 让文件格式化（有缩进），方便人类阅读
#     # ensure_ascii=False 防止中文（如果有的话）被转义成乱码
#     json.dump(prompt_data_list, f, indent=2, ensure_ascii=False)

# print("Save complete (with labels).")

In [11]:
# ==========================================
# 3. 构建 Prompt (System vs User)
# ==========================================

# System Prompt: 定义规则、类比和输出格式
system_prompt = """You are an expert bioinformatics assistant capable of linguistic transfer learning.

1. The Concept:
In English, a sentence can be rearranged structurally but keep the same meaning (Paraphrase). Or, it can be scrambled to lose its logic (Adversarial).

2. The Analogy:
* **Homologous Proteins** are like **Paraphrases**: They have different sequences due to evolution, but they fold into the same structure and function.
* **Non-Homologous/Random Proteins** are like **Adversarial Sentences**: They look like proteins, but their internal structural logic is broken.

3. Output Requirements:
* I will provide a JSON list of protein pairs.
* You must return a RAW JSON object containing a list of results.
* The format must be strictly: `[{"id": 1, "prediction": "Homologous"}, {"id": 2, "prediction": "Non-Homologous"}, ...]`
* Do NOT provide explanations. Just the JSON array.
"""

# User Prompt: 提供具体数据
user_prompt = f"""Here is the JSON list of 100 protein pairs to analyze.
Using your intuition about "sequence syntax" and "structural integrity," determine if each pair is "Homologous" or "Non-Homologous".

Data:
{json.dumps(prompt_data_list, indent=2)}
"""

In [12]:
# ==========================================
# 4. 执行本地推理 (Local Inference)
# ==========================================
print("-" * 30)
print(f"Running inference with {MODEL_ID}...")

# 1. 构建对话格式
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

# 2. 应用 Chat Template (这是 Llama 3 听懂指令的关键)
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

# 3. 生成配置
print("Generating response (this may take a while for batch processing)...")

try:
    with torch.no_grad():
        outputs = model.generate(
            input_ids,
            max_new_tokens=4096,   # 给足够的 Tokens 生成 JSON (100条数据需要较长输出)
            eos_token_id=terminators,
            do_sample=True,        # 采样模式
            temperature=0.1,       # 低温，保证 JSON 格式稳定
            top_p=0.9,
        )

    # 4. 解码输出
    # 只截取新生成的部分 (input_ids 长度之后的内容)
    response = outputs[0][input_ids.shape[-1]:]
    full_content = tokenizer.decode(response, skip_special_tokens=True)

    print("Response received.")
    print(f"Response snippet: {full_content[:200]}...")

except Exception as e:
    print(f"Inference Failed: {e}")
    full_content = ""

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


------------------------------
Running inference with meta-llama/Llama-3.1-8B-Instruct...
Generating response (this may take a while for batch processing)...
Response received.
Response snippet: [{"id": 1, "prediction": "Homologous"}, {"id": 2, "prediction": "Homologous"}, {"id": 3, "prediction": "Homologous"}, {"id": 4, "prediction": "Homologous"}, {"id": 5, "prediction": "Non-Homologous"}, ...


In [13]:
# import json

# # 定义文件名
# output_filename = "llm_raw_output.json"
# label_map = {"Homologous": 1, "Non-Homologous": 0}

# print(f"Processing and saving to {output_filename}...")

# # 1. 简单清洗：去掉可能存在的 markdown 符号 (```json 和 ```)
# cleaned_content = full_content.replace("```json", "").replace("```", "").strip()

# # 2. 解析成 Python 列表
# data_list = json.loads(cleaned_content)

# # 3. 循环添加 pred_label
# for item in data_list:
#     # 直接根据 prediction 查表得到 1 或 0
#     item["pred_label"] = label_map[item["prediction"]]

# # 4. 写入文件
# with open(output_filename, "w", encoding="utf-8") as f:
#     json.dump(data_list, f, indent=2, ensure_ascii=False)

# print("Save complete.")

In [14]:
# ==========================================
# 5. 解析结果与评估
# ==========================================
def parse_llm_json(text):
    """提取并解析 JSON"""
    try:
        # 寻找 JSON 数组 [...]
        match = re.search(r"\[.*\]", text, re.DOTALL)
        if match:
            return json.loads(match.group(0))
        return json.loads(text)
    except Exception:
        return []

# 解析
predictions_list = parse_llm_json(full_content)

# 映射与计算
label_map = {"Homologous": 1, "Non-Homologous": 0}
y_true = []
y_pred = []

print("-" * 30)
if not predictions_list:
    print("Failed to parse JSON from model response.")
else:
    print(f"Parsed {len(predictions_list)} predictions.")
    
    for item in predictions_list:
        p_id = item.get('id')
        p_str = item.get('prediction')
        
        # 确保 ID 存在且预测值有效
        if p_id in id_to_ground_truth and p_str in label_map:
            y_true.append(id_to_ground_truth[p_id])
            y_pred.append(label_map[p_str])

    # 输出最终指标
    if y_true:
        acc = accuracy_score(y_true, y_pred)
        print(f"\nFinal Accuracy: {acc:.2%}")
        print("\nClassification Report:")
        print(classification_report(y_true, y_pred, target_names=["Non-Homologous (0)", "Homologous (1)"]))
        
        # 保存结果用于后续分析
        result_log = {
            "model": MODEL_ID,
            "accuracy": acc,
            "predictions": predictions_list
        }
        print(acc)
        # with open("doubao_result.json", "w") as f:
        #     json.dump(result_log, f, indent=2)
        #     print("Results saved to 'doubao_result.json'")
    else:
        print("No valid matching IDs found between Prompt and Response.")

------------------------------
Parsed 100 predictions.

Final Accuracy: 82.00%

Classification Report:
                    precision    recall  f1-score   support

Non-Homologous (0)       0.86      0.76      0.81        50
    Homologous (1)       0.79      0.88      0.83        50

          accuracy                           0.82       100
         macro avg       0.82      0.82      0.82       100
      weighted avg       0.82      0.82      0.82       100

0.82
